Wahlpflichtfach Künstliche Intelligenz I: Praktikum

---

# 08 Scikit-learn - Data Preparation

__Scikit-learn__ (auch als __sklearn__ bekannt) ist ein Open-Source Software-Bibliotheke zum maschinellen Lernen in Python. Sie erfreut sich großer Beliebtheit und wird aktiv gewartet. Die Bibliothek bietet verschiedene Klassifikations-, Regressions- und Clustering-Algorithmen an. Darüber hinaus sind auch Algorithmen zur Modellauswahl, Dimensionsreduktion und Datenvorverarbeitung in sklearn enthalten. 

In diesem Noteboob beschäftigen wir uns (erneut) mit der Datenvorverarbeitung (Data Preparation) und gehen dabei auf die folgenden Themen ein:
- Imputation
- Skalierung
- Dimensionreduktion
- Pipelines
- Feature Union
- Column Transformations

Die Dokumentation zu scikit-learn ist [hier](https://scikit-learn.org/stable/index.html) zu finden.

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Imputation
Unter Imputation ist das Vervollständigen von fehlenden Werten (NaNs) zu verstehen. Wie in pandas auch gibt es verschiedene Methoden um fehlende Werte in sklearn zu ersetzen. Weitere Informationen sind [hier](https://scikit-learn.org/stable/modules/impute.html) zu finde.

In [2]:
nan_data = np.array([[1, 2], [np.nan, 3], [7, 6]])

### Eindimensionale Imputation
In der eindimensonalen Imputation werden die Werte je Spalte ersetzt. Die Klasse [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer) bietet dafür grundlegende Strategien. Fehlende Werte können mit einem bereitgestellten konstanten Wert oder mit statistischen Werten (Mittelwert, Median oder häufigster Wert) jeder Spalte, in der sich die fehlenden Werte befinden, ersetzt werden. Diese Klasse erlaubt auch verschiedene Kodierungen für fehlende Werte.

In [3]:
from sklearn.impute import SimpleImputer

# define different imputer
mean_imputer = SimpleImputer()
zero_imputer = SimpleImputer(strategy='constant', fill_value=0)

In [ ]:
# First the imputer has to be fitted to the data, so either call first fit and then transform 
# or call fit_transform to do it in one step
mean_imputer.fit(nan_data)
mean_imputer.transform(nan_data)

In [ ]:
zero_imputer.fit_transform(nan_data)

In [ ]:
different_nan_data = np.array([[np.nan, 5], [8, 2], [6, 6]])
mean_imputer.transform(different_nan_data)

__Brainstorming:__
<details>
<summary>Wieso wurde np.nan durch 4 ersetzt?</summary>
Da der mean_inputer zuvor auf den anderen Daten "trainiert" wurde. 
</details>

<details>
<summary>Wann kann dieses Verhalten von Vorteil sein?</summary>
Ein Vorteil ist es, dass die Ersetzungs-Strategien bzw. exakten Werte, die während des Trainings verwendet wurden, auch zur Test-Zeit bzw. im Live-Betrieb verwendet werden können. 
</details>

In [ ]:
zero_imputer.transform(different_nan_data)

Es ist auch möglich andere Werte als `np.nan` zu ersetzen. 

In [ ]:
fischers_fritz = [['Fischers', '', 'fischt', 'frische', 'Fische'],
                  ['Frische', 'Fische', 'fischt', 'Fischers', '']]

string_imputer = SimpleImputer(missing_values='', strategy='constant', fill_value='Fritz')
string_imputer.fit_transform(fischers_fritz)          

### Mehrdimensionale Variante
Die mehrdimensionalen Variante ist dahingegen deutlich anspruchsvoller. Grob zusammengefasst wird jeder fehlende Wert als Funktion anderer Merkmale modelliert und diese Schätzung zur Imputation verwendet. Dieser Vorgang wird dann einige Male wiederholt,, bevor die finalen Ersetzungen vorgenommen werden. Dieses Verhalten wird vom [IterativeImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html#sklearn.impute.IterativeImputer) implementiert. 

__Achtung:__ Diese Klasse ist noch experimentell 

In [9]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

Zuerst erstellen wir uns ein kleines Dummy-Dataset, wo die Werte der einzelnen Spalten eine klare Beziehung zueinander haben. 

In [ ]:
x = np.arange(1, 11, dtype="float")
y = x * x 
data = np.array([x, y])
data[(0, 1)] = np.nan
data[(0, 6)] = np.nan
data[(1, 3)] = np.nan
data[(1, 9)] = np.nan
data = data.T
data

Anschließend können wir wieder mit `fit_transform` die Daten ersetzen.

In [ ]:
IterativeImputer().fit_transform(data)

Die Ersetzungen des `SimpleImputer`s sehen hingegen folgendermaßen aus.

In [ ]:
SimpleImputer().fit_transform(data)

__Brainstorming:__
<details>
    <summary>Welche Vorteile bringt die mehrdimensionale Variante?</summary>
    Ein großer Vorteil ist, dass die nicht vorhandenen Daten abhängig von  anderen Daten ersetzt werden. Dies ist häufig besser, da es Abhängigkeiten zwischen den Daten geben kann. Beispiel: Größe und Gewicht von Personen.
</details>

In den folgenden drei Codezeilen, wird die iterative Arbeitsweise des Algorithmus sichtbar.

In [ ]:
IterativeImputer(max_iter=1).fit_transform(data)

In [ ]:
IterativeImputer(max_iter=2).fit_transform(data)

In [ ]:
IterativeImputer(max_iter=3).fit_transform(data)

## Skalierung

### Standardisierung
Unter Standardisiserung ist eine Transformation der Eingabedaten zu verstehen, so dass die resultierenden Daten eine Mittelwert von 0 und eine Varianz von 1 haben. Die resultierenden Daten sind also normalverteilt. Dies ist besonders hilfreich, wenn alle Variablen unterschiedlich skaliert sind. 

__Beispiel:__ Ein Datensatz enthält die Werte Größe in Metern und Gewicht in Kilogramm von Menschen. Die Varianz der Variablen wird deutlich unterschiedlich sein, da die Größe sich auf einer Skala von 0,1 m bis 2,8 m befindet und das Gewicht auf einer Skala von 0,5 kg bis 600 kg.

In sklearn wird für die Standardisierung die Klasse [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) verwendet. 

In [ ]:
from sklearn.preprocessing import StandardScaler

scaling_data = np.array([[1.79, 79.5], 
                         [1.60, 53.2], 
                         [2.59, 150], 
                         [1.73, 70.7], 
                         [1.50, 46.7], 
                         [1.75, 113.0], 
                         [1.93, 247.2]])
print(f"Mittelwerte pro Feature: {scaling_data.mean(axis=0)}")
print(f"Standardabweichung pro Feature: {scaling_data.std(axis=0)}")

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(scaling_data)
scaled_data

In [ ]:
print(f"Mittelwerte pro Feature: {scaled_data.mean(axis=0)}")
print(f"Standardabweichung pro Feature: {scaled_data.std(axis=0)}")

__Brainstorming:__
<details>
<summary>Wieso ist der Mittelwert nicht bei 0?</summary>
Das sind Berechnungsfehler, die vernachlässigt werden können. e-17 ist eine verdammt kleine Zahl.
</details>

Wenn die Daten nicht zentriert werden sollen, kann dies mit dem Parameter `with_mean=False` verhindert werden. 

In [ ]:
not_center_scaler = StandardScaler(with_mean=False)
non_centric_data = not_center_scaler.fit_transform(scaling_data)

print(f"Mittelwerte pro Feature: {non_centric_data.mean(axis=0)}")
print(f"Standardabweichung pro Feature: {non_centric_data.std(axis=0)}")
non_centric_data

Ebenso ist es möglich die Daten nicht zu skalieren, um die Varianz zu behalten. Dafür muss der Konstruktor mit dem Parameter `with_std=False` aufgrufen werde.

In [ ]:
not_scaling_scaler = StandardScaler(with_std=False)
non_scaled_data = not_scaling_scaler.fit_transform(scaling_data)

print(f"Mittelwerte pro Feature: {non_scaled_data.mean(axis=0)}")
print(f"Standardabweichung pro Feature: {non_scaled_data.std(axis=0)}")
non_scaled_data

### Skalierung der Features in einem bestimmten Bereich
Alternativ kann die Skalierung von Featuren so erfolgen, dass sie zwischen einem vorgegebenen Minimal- und Maximalwert liegen, oft zwischen Null und Eins, oder so, dass der maximale Absolutwert jedes Features auf Einheitsgröße skaliert wird. Dies kann mit [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#) bzw. [MaxAbsScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html#) erreicht werden. 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
min_max_data = np.array([[-1, 2], 
                         [-0.5, 6], 
                         [0, 10], 
                         [1, 18]])

zero_one_scaler = MinMaxScaler()
zero_one_scaler.fit_transform(min_max_data)

Um den Bereich zu ändern kann im Konstruktor der Parameter `feature_range` spezifiziert werden.

In [ ]:
minus_one_one_scaler = MinMaxScaler(feature_range=(-1, 1))
minus_one_one_scaler.fit_transform(min_max_data)

In [ ]:
from sklearn.preprocessing import MaxAbsScaler
max_abs_data = np.array([[ 1., -1.,  2.],
                         [ 2.,  0.,  0.],
                         [ 0.,  1., -1.]])

max_abs_scaler = MaxAbsScaler()
max_abs_scaler.fit_transform(max_abs_data)

### Probleme mit Ausreißern
Die drei vorgestellten Klassen können bei der Skalierung nicht besonders gut mit Ausreißern (outlier) umgehen. Dieses Problem wird [hier](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py) dargestellt.

Ein Skaler, der mit Ausreißern funktioniert ist der [RobustScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html). 

In [ ]:
from sklearn.preprocessing import RobustScaler
outlier_data = np.array([[2, 4, 1, 27, 3, 4, 1, 3, 3, 2],
                [100, 92, 87, 94, 95, 83, 177, 84, 99, 89]]).T

robust_scaler = RobustScaler(quantile_range=(25, 75))
robust_scaler.fit_transform(outlier_data)

In [ ]:
robust_scaler.scale_

In [ ]:
robust_scaler.center_

Im Vergleich dazu würde der StandardScaler die Daten folgendermaßen skalieren.

In [ ]:
StandardScaler().fit_transform(outlier_data)

__Aufgabe:__ 
<details>
<summary>Wie werden die neuen Werte berechnet?</summary>
Zuerst müssen der Median und die Quantile pro Spalte bestimmt werden. Dann können die neuen Werte wie folgt berechnet werden:
    $$x_{neu} = \frac{x_{alt} - median}{quantile_{upper} - quantil_{lower}}$$
</details>

## Dimensionsreduktion
Unter Dimensionsreduktion ist die Reduzierung der Anzahl der Zufallsvariablen zu verstehen. Dies kann sowohl für die Visualisierung als auch fürs maschinelle Lernen hilfreich sein.

Im Folgendem werden wir den Iris-Datensatz (Iris = Schwertlilie) verwenden. Er besteht aus 4 Features (Sepal Länge, Sepal Breite, Petal Länge und Petal Breite) und hat 3 Klassen. 

In [ ]:
iris = datasets.load_iris(as_frame=True)
iris_df = iris.frame
iris_df['Label'] = iris.target_names[iris_df['target']]
iris_df = iris_df.drop('target', axis=1)

sns.set_theme(style="whitegrid")
sns.pairplot(iris_df, hue="Label")

### Principial Component Analyse (PCA)
PCA wird verwendet, um einen multidimensionalen Datensatz in eine Menge von aufeinanderfolgenden orthogonalen Komponenten zu zerlegen, die einen maximalen Anteil der Varianz erklären. Wie auch die vorherigen Algorithmen ist [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) als Transformer-Objekt implementiert. Das heißt es muss erst mit `fit()` trainiert werden. Anschließend kann es dann auf die Daten angewandt werden und die $n$ Feature finde, die den größten Anteil der Varainz in den Daten erklären.

In [ ]:
from sklearn.decomposition import PCA

# we want to have the two components with the highest variance
pca = PCA(n_components=2)
pca.fit(iris.data)
reduced_iris = pca.transform(iris.data)

# show the two components with the highest variance
ax_pca = sns.scatterplot(x=reduced_iris[:, 0], y=reduced_iris[:, 1], hue=iris_df['Label'])
ax_pca.set_xlabel('Dimension 1')
ax_pca.set_ylabel('Dimension 2')

__Brainstorming:__
<details>
<summary>Was könnte eine Schwäche von PCA im Kontext einer Klassifikationsaufgabe sein?</summary>
PCA versucht die größte Varianz in der Gesamtheit der Daten zu finden. Dabei beachtet PCA aber nicht die Varianz zwischen den einzelnen Klassen. Dies kann dazu führen, dass suboptimale Features für die Klassifizeirung ausgewählt werden. 
</details>

### Linear Discriminant Analysis (LDA)
[LDA]() verwendet im Gegensatz zu PCA auch die Klassenzugehörigkeit. Dabei versucht es die Features zu ermitteln, die den größten Einfluss auf die Varianz zwischen den Klassen haben. Da LDA die Klassenzugehörigkeit verwendet, handelt es sich dabei um eine supervised Methode. 

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(n_components=2)
lda.fit(iris.data, iris.target)
lda_reduced_iris = lda.transform(iris.data)

# show the two components with the highest variance between classes
ax_lda = sns.scatterplot(x=lda_reduced_iris[:, 0], y=lda_reduced_iris[:, 1], hue=iris_df['Label'])
ax_lda.set_xlabel('Dimension 1')
ax_lda.set_ylabel('Dimension 2')

### Vergleich von PCA und LDA

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,6))
sns.scatterplot(x=reduced_iris[:, 0], y=reduced_iris[:, 1], hue=iris_df['Label'], ax=ax1)
ax1.set_xlabel('Dimension 1')
ax1.set_ylabel('Dimension 2')
ax1.set_title('PCA')
sns.scatterplot(x=lda_reduced_iris[:, 0], y=lda_reduced_iris[:, 1], hue=iris_df['Label'], ax=ax2)
ax2.set_xlabel('Dimension 1')
ax2.set_ylabel('Dimension 2')
ax2.set_title('LDA')
fig.suptitle('Comparison of PCA and LDA')
fig.tight_layout()

## Pipelines
[Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) kann verwendet werden, um mehrere Kalkulatoren zu einem zu verketten. Dies ist nützlich, da es oft eine feste Abfolge von Schritten bei der Verarbeitung der Daten gibt, z. B. Merkmalsauswahl, Normalisierung und Klassifizierung. Pipeline dient hier mehreren Zwecken:

1) __Bequemlichkeit und Verkapselung__
> Sie müssen `fit` und `predict` nur einmal auf Ihren Daten aufrufen, um eine ganze Sequenz von Kalkulatoren anzupassen.

2) __Gemeinsame Parameterauswahl__
> Sie können eine Rastersuche (Grid Search) über die Parameter aller Kalkulatoren in der Pipeline auf einmal durchführen. Darauf gehen wir später weiter ein.

3) __Sicherheit__
> Pipelines helfen dabei, ein Durchsickern von Statistiken aus Ihren Testdaten in das trainierte Modell bei der Kreuzvalidierung (darauf gehen wir auch später noch ein) zu vermeiden, indem sichergestellt wird, dass die gleichen Stichproben zum Trainieren der Transformatoren und Vorhersager verwendet werden.

Alle Kalkulatoren in einer Pipeline, außer dem letzten, müssen Transformatoren sein (d. h. sie müssen eine `transform`-Methode haben). Der letzte Kalkulatoren kann ein beliebiger Typ sein (Transformator, Klassifikator usw.).

### Konstruktor
Die [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) wird mit einer Liste von (Schlüssel, Wert)-Paaren aufgebaut, wobei der Schlüssel eine Zeichenkette ist, die den Namen enthält, den Sie diesem Schritt geben möchten, und der Wert ein Kalkulator ist.

In [ ]:
from sklearn.pipeline import Pipeline

# first, create a dictionary with all the estimators we want to have in the pipeline, each step needs a name
estimators = [('imputer', SimpleImputer()), ('scaler', StandardScaler())]

# then, pass the estimators to the Pipeline constructor
pipe = Pipeline(estimators)
pipe

In [ ]:
# if you don't want to name the estimators explicit you can use the make_pipeline function
from sklearn.pipeline import make_pipeline
make_pipeline(SimpleImputer(), StandardScaler())

### Parameter der Kalkulatoren setzen
Auf die Parameter der Kalkulatoren in der Pipeline kann mit der Syntax `<estimator>__<parameter>` (_2 Unterstriche_) zugegriffen werden.

In [ ]:
pipe.set_params(imputer__strategy='constant', imputer__fill_value=0)

Nachdem die Parameter gesetzt wurden können wir die Pipeline mit dummy-Werten testen. 

In [ ]:
pipe_data = [[1, 4], [5, 8], [np.nan, 5], [3, np.nan]]
pipe.fit_transform(pipe_data)

Um andere Parameter zu verwenden, müssen diese wieder mit `set_params()` gesetzt werden.

In [ ]:
# reset to default parameters
pipe.set_params(imputer__strategy='mean', imputer__fill_value=None)

In [ ]:
pipe.fit_transform(pipe_data)

Des Weiteren ist es möglich auch einzelne Schritte komplett zu ersetzen. Dazu wird ebenfalls wieder `set_params()` verwendet und der Name des Schrittes angegeben, der ersetzt werden soll.

In [ ]:
pipe.set_params(scaler=MaxAbsScaler())

In [ ]:
pipe.fit_transform(pipe_data)

Wenn ein Schritt in der Pipeline nicht mehr ausgeführt werden soll, kann dies durch das Übergeben des Wertes `'passthrough'` erreicht werden.

In [ ]:
pipe.set_params(imputer='passthrough')

In [ ]:
pipe.fit_transform(pipe_data)

## Feature Union
[FeatureUnion](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html) kombiniert mehrere Transformator-Objekte zu einem neuen Transformator, der deren Ausgabe kombiniert. Eine FeatureUnion nimmt eine Liste von Transformator-Objekten auf. Während der Anpassung wird jeder von ihnen unabhängig an die Daten angepasst. Die Transformatoren werden parallel angewendet und die von ihnen ausgegebenen Feature-Matrizen werden nebeneinander zu einer größeren Matrix verkettet.

### Konstruktor
Eine FeatureUnion wird aus einer Liste von (Schlüssel, Wert)-Paaren aufgebaut, wobei der Schlüssel der Name ist, den Sie einer gegebenen Transformation geben möchten und der Wert ein Kalkulator ist.

In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import KernelPCA

estimators = [('linear_pca', PCA()), ('kernel_pca', KernelPCA())]
combined = FeatureUnion(estimators)
combined

Auch hier ist es wieder möglich die Schritte nicht explizit zu benennen. Dafür kann die Methode `make_union()` verwendet werden.

In [ ]:
from sklearn.pipeline import make_union
make_union(PCA(), KernelPCA())

In [ ]:
combined_data = [[1, 4, 3], [5, 8, 7], [2, 5, 4], [3, 6, 5]]
combined.fit_transform(combined_data)

### Parameter der Transformer setzen
Auch hier können die Parameter der Transformer wieder über die Methode `set_params()` gesetzt werden. 

In [ ]:
combined.set_params(linear_pca__n_components=3)

Um einen Schritt zu ignorieren muss an den entsprechenden Schrittname der Wert `'drop'` übergeben werden.

In [ ]:
combined.set_params(kernel_pca='drop')

## Column Transformations
Viele Datensätze enthalten Merkmale verschiedener Typen, z. B. Text, Fließkommazahlen und Datumsangaben, wobei jeder Merkmalstyp separate Vorverarbeitungs- oder Featureextraktionsschritte erfordert. Oft ist es am einfachsten, die Daten vor der Anwendung von scikit-learn-Methoden vorzuverarbeiten, zum Beispiel mit __Pandas__. 

Allerdings kann die Verarbeitung der Daten vor der Übergabe an scikit-learn aus einem der folgenden Gründe problematisch sein:
1) Das Einbeziehen von Statistiken aus Testdaten in die Präprozessoren macht die Ergebnisse der Kreuzvalidierung unzuverlässig (bekannt als Datenleck), zum Beispiel im Fall von Skalierern oder der Imputierung fehlender Werte.

2) Sie möchten möglicherweise die Parameter der Vorverarbeitung in eine Parametersuche einbeziehen.

Der [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) hilft bei der Durchführung verschiedener Transformationen für verschiedene Spalten der Daten, innerhalb einer Pipeline, die sicher vor Datenlecks ist und die parametrisiert werden kann.

### Dummy-Daten erstellen
Zuerst erstellen wir uns ein Datensatz der neben nummerischen Daten auch Text enthält.

In [47]:
book_data_raw = pd.DataFrame(
    {
        'city': ['London', 'London', 'Paris', 'Sallisaw'],
        'title': ["His Last Bow", "How Watson Learned the Trick", "A Moveable Feast", "The Grapes of Wrath"],
        'expert_rating': [5, 3, 4, 5],
        'user_rating': [4, 5, 4, 3]
    }
)

__Brainstorming:__

<details>
<summary>Wie können wir die Textdaten verwenden?</summary>
Um die Textdaten fürs maschinelle Lernen zu verwenden müssen diese in Zahlen umgewandelt werden. In diesem Fall kann die Stadt beispielsweise One-Hot encoded werden und für den Titel können wir die vorkommenden Wörter zählen (im Seminar werden wir noch genauer darauf eingehen).
</details>

### Konstruktor
Wir wollen auf die Spalte `'city'` den [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) anwenden und auf die Spalte `'title'` den [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer). 

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
column_trans = ColumnTransformer([('city_category', OneHotEncoder(dtype='int'),['city']),
                                  ('title_bow', CountVectorizer(), 'title')])

column_trans.fit(book_data_raw) 

Die Namen der Features bekommt man über die Methode `get_feature_names()`.

In [ ]:
column_trans.get_feature_names_out()

In [ ]:
column_trans.transform(book_data_raw).toarray()

Alle Features, die nicht an einen Transformator übergeben wurden, werden defaultmäßig nicht mit ausgegeben. Um dieses Verhalten zu verändern, kann dem Konstruktor ein Wert für den Parameter `remainder` übergeben werden.  

In [ ]:
column_trans = ColumnTransformer([('city_category', OneHotEncoder(dtype='int'),['city']),
                                  ('title_bow', CountVectorizer(), 'title')],
                                remainder='passthrough')

column_trans.fit(book_data_raw)

In [ ]:
column_trans.get_feature_names_out()

In [ ]:
column_trans.transform(book_data_raw)

Auch hier gibt es wieder die Möglichkeit, die Namen für die einzelnen Schritte nicht explizit zu vergeben. Dafür kann die Methode `make_column_transformer()` verwendet werden. Außerdem kann man auch einen Transformer an den Parameter `remainder` übergeben. Dieser wird dann auf alle übrigen Spalten angewandt.  

In [ ]:
from sklearn.compose import make_column_transformer
column_trans = make_column_transformer(
    (OneHotEncoder(), ['city']),
    (CountVectorizer(), 'title'),
    remainder=MinMaxScaler()
)
column_trans

In [ ]:
column_trans.fit_transform(book_data_raw)

## Generelle Funktionsweise
Wie Ihr bestimmt bereits bemerkt habt sind die Schritte immer wieder die gleichen:
1. __Konstruktor:__ Erstellen des Algorithmus. Die meisten Algorithmen können in diesem Schritt parametrisiert werden.
2. __fit():__ Training des Algorithmus anhand der übergebenden Daten.
3. __transform():__ Mit der `transform()`-Methode kann der Agorithmus anschließend die Transformationen für die übergebenen Werte druchführen.

---

Wahlpflichtach Künstliche Intelligenz I: Praktikum 